# Prompt

千帆提供了 Prompt 管理功能，可以快速地使用平台预置的优质 Prompt，或者保存用户自定义的 Prompt。SDK 也为用户快速使用 Prompt 提供了辅助。

Prompt 相关功能需要使用 Access Key 和 Secret Key 进行鉴权，获取方式参见 [文档](https://cloud.baidu.com/doc/Reference/s/9jwvz2egb)。

In [1]:
import os
import qianfan

os.environ["QIANFAN_ACCESS_KEY"] = "your access key"
os.environ["QIANFAN_SECRET_KEY"] = "your secret key"

可以通过这一方式导入 Prompt

In [1]:
from qianfan.components import Prompt

## 快速使用

平台上预置的 Prompt 以及用户自定义的模型都可以在 [千帆控制台](https://console.bce.baidu.com/qianfan/prompt/template) 获得，之后可以在 SDK 中用 Prompt 的名称快速获取 Prompt 对象。

In [5]:
p = Prompt(name="区域美食推荐")

之后就可以调用 `render` 方法对 Prompt 模版进行填充，得到最终的文本。

In [6]:
# 第二个参数是 negative prompt，文生文场景下为空，所以用 _ 省略
prompt, _ = p.render(region="上海")
print(prompt)

作为游客，告诉我上海必吃的10大美食


也可以通过如下方式获得 Prompt 的一些元信息

In [7]:
print(p.template)
print(p.variables)

作为游客，告诉我{region}必吃的10大美食
['region']


SDK 也支持文生图类型的 Prompt，可以通过文生图类的 Prompt 名称进行初始化

In [5]:
txt2img_prompt = Prompt(name="角色设计")

prompt, negative_prompt = txt2img_prompt.render()
print(prompt)
print(negative_prompt)

white hair female, close up character design, multiple concept designs, concept design sheet, white background, style of Yoshitaka Amano
(worst quality, low quality:1.4), deformed iris, deformed pupils, (deformeddistorted, disfigured:1.3), cropped, out of frame.poorly drawn, bad anatomy, wrong anatomy, extralimb, missing limb, floating limbs


Prompt 的场景可以通过 `scene_type` 属性获得

In [6]:
from qianfan.consts import PromptSceneType

txt2img_prompt.scene_type == PromptSceneType.Text2Image

True

借助 Prompt，我们就可以快速地完成一些任务，比如：

In [16]:
p = Prompt(name="区域美食推荐")

def recommend_food(region):
    prompt, _ = p.render(region=region)
    r = qianfan.Completion().do(prompt)
    return r['result']

print(recommend_food("上海"))

作为游客，上海必吃的十大美食包括：

1. 小笼包：上海的小笼包最为著名，皮薄、汁多、肉嫩、味鲜，是上海本地的经典小吃。
2. 生煎馒头：上海特色传统小吃，上海人一般叫它“生煎”或“生煎包子”。生煎馒头可以说是上海点心的代表之一，非常适合当早餐，不仅好吃，而且非常有营养。
3. 蟹壳黄：是上海本地的一种特色小吃，形状有点像蟹壳，颜色金黄，香酥可口。
4. 油条糍饭团：作为早点或夜宵的小吃，糯米糍软糯有嚼劲，夹着油条一起吃别有一番风味。
5. 糖醋排骨：是上海市传统名菜之一，外酥里嫩、酸甜可口，深受许多人喜爱。
6. 糟卤肉：糟卤肉是上海市民最喜爱的佐餐小菜之一，糟卤肉以其独特的味道而受到广泛欢迎。
7. 三鲜砂锅：这是一道非常具有上海特色的家常菜，将鸡肉、猪肉、鱼肉等原料炖煮在一起，鲜美可口。
8. 响油鳝糊：是上海市常见的地方特色小吃，鳝糊表面浇上了一层热油，香味四溢。
9. 冷面：冷面是上海夏天非常受欢迎的小吃之一，不仅美味可口，而且清凉解暑。
10. 鱼丸汤：鱼丸汤是一道非常具有上海本地特色的汤品，鱼丸鲜美Q弹，汤底清淡，非常适合作为早餐或午餐。

除了以上美食，上海还有许多其他美食等待你去探索和品尝。在品尝美食的同时，也请注意适量食用，避免过度摄入热量和脂肪。


## 本地 Prompt

如果不希望使用平台上的模型，只希望本地尝试，也可以通过 Prompt 对象实现，初始化时仅传入 `template` 字段即可，如果需要设置 Prompt 的 `name`，则还需要加上 `mode="local"`，后续使用方法与上述一致。

In [2]:
p = Prompt(
    template="本地 prompt {var1}",
)

prompt, _ = p.render(var1="hello")
print(prompt)

本地 prompt hello


还可以传入更多参数，创建一个更为复杂的本地 Prompt，比如下面创造了一个场景为文生图的 Prompt，并且给定了分别设置了正向和负向 Prompt，并且采用 `(())` 作为标识符。

In [8]:
prompt = Prompt(
    # 如下两个参数在 local 模式下可选
    # 但如果传入 name，那么必须设置 mode 才能设置为本地模式，否则会从服务器拉取
    # name="txt2img",
    # mode="local",
    template="txt2img template ((v1))",
    scene_type=PromptSceneType.Text2Image,
    negative_template="negative template ((v3))",
    identifier="(())",
)

## 上传/更新 Prompt

对于本地 Prompt，还可以将其上传保存至平台，方便后续快速使用，上传仅需要调用 `upload` 方法即可。

In [9]:
p = Prompt(
    template="本地 prompt {var1}",
)

# 对于平台上的 prompt 来说，name 是必须的，因此上传前必须先设置
p.name = "cookbook_prompt"
p.upload()

p.id

11945

上传完成后，就可以通过 `id` 属性获得在平台上的 id。

后续还可以本地对 Prompt 进行更新，并用 `upload` 方法更新至平台上，对于 `remote` 的 Prompt 也同样如此。

In [10]:
p.set_template("新的 Prompt {new_var}")
p.upload()

print(p.variables)
print(p.render(new_var="hello"))

['new_var']
('新的 Prompt hello', None)


## 删除 Prompt

SDK 提供了 `delete` 方法，可以快速删除平台上的 Prompt。

In [11]:
p = Prompt(name="cookbook_prompt")
p.delete()

如果没有抛出异常，那么这条 Prompt 已经从平台上移除。

## 保存/读取 Prompt

通常在进行 Prompt 调优后，需要保存 Prompt 以便后续使用，SDK 也提供了 `save_to_file` 方法，可以将 Prompt 保存保存至本地。 

In [12]:
p = Prompt(template="这是一个用于{usage}的 Prompt", mode="local")
p.save_to_file("test_prompt.tpl")

而在再次使用时，只需要通过 `from_file` 方法即可读取 Prompt。

如果希望调整 Prompt，可以直接对模版文件修改，从而既可以避免在代码中出现冗长的模版字符串，也可以因调整模版而反复修改代码。

In [13]:
p = Prompt.from_file("test_prompt.tpl")
prompt, _ = p.render(usage="测试")
print(prompt)

这是一个用于测试的 Prompt
